In [ ]:
from importlib.metadata import version
import nltk
import tensorflow
import summa
import pandas as pd


print(nltk.__version__)
print(tensorflow.__version__)
print(pd.__version__)
print(version('summa'))

3.9.1
2.18.0
2.2.2
1.2.0


In [ ]:

pip install summa

In [ ]:
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv", filename="news_summary_more.csv")
data = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')

In [ ]:
data.head(10)

,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...
5,Rahat Fateh Ali Khan denies getting notice for...,Pakistani singer Rahat Fateh Ali Khan has deni...
6,"India get all out for 92, their lowest ODI tot...",India recorded their lowest ODI total in New Z...
7,Govt directs Alok Verma to join work 1 day bef...,Weeks after ex-CBI Director Alok Verma told th...
8,Called PM Modi 'sir' 10 times to satisfy his e...,Andhra Pradesh CM N Chandrababu Naidu has said...
9,"Cong wins Ramgarh bypoll in Rajasthan, takes t...",Congress candidate Shafia Zubair won the Ramga...


In [ ]:
# STEP 1: 라이브러리 임포트 및 NLTK stopwords 다운로드
import nltk
import pandas as pd
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Lambda, Attention
from summa.summarizer import summarize

nltk.download('stopwords')

# STEP 2: 데이터 로드 및 정제 (본문/헤드라인 모두)
data = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub('"','', text)
    text = re.sub(r"'s\b","", text)
    text = re.sub("[^a-zA-Z]", " ", text)
    text = re.sub('[m]{2,}', 'mm', text)
    text = re.sub('\s+', ' ', text)
    return text.strip()

data['clean_text'] = data['text'].apply(clean_text)
data['clean_headlines'] = data['headlines'].apply(clean_text)

# STEP 3: 텍스트 토크나이징 및 시퀀스 패딩
text_vocab = 8000
head_vocab = 2000
max_text_len = 100
max_head_len = 10

text_tokenizer = Tokenizer(num_words=text_vocab)
text_tokenizer.fit_on_texts(data['clean_text'])
text_seq = text_tokenizer.texts_to_sequences(data['clean_text'])
text_pad = pad_sequences(text_seq, maxlen=max_text_len, padding='post')

head_tokenizer = Tokenizer(num_words=head_vocab, oov_token='OOV')
head_tokenizer.fit_on_texts(data['clean_headlines'])
head_seq = head_tokenizer.texts_to_sequences(data['clean_headlines'])
head_pad = pad_sequences(head_seq, maxlen=max_head_len, padding='post')

x_train, y_train = text_pad, head_pad

# STEP 4: 어텐션이 포함된 Seq2Seq 모델 설계
emb_dim = 128
latent_dim = 256

# 인코더
encoder_input = Input(shape=(max_text_len,))
enc_emb = Embedding(text_vocab, emb_dim)(encoder_input)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# 디코더
decoder_input = Input(shape=(max_head_len,))
dec_emb = Embedding(head_vocab, emb_dim)(decoder_input)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

# 어텐션 메커니즘 적용
attention = Attention()
encoder_outputs_expanded = Lambda(lambda x: tf.expand_dims(x, 1))(encoder_outputs)
context_vector = attention([decoder_outputs, encoder_outputs_expanded])
decoder_concat_input = Lambda(lambda x: tf.concat(x, axis=-1))([decoder_outputs, context_vector])

# 출력층
decoder_dense = Dense(head_vocab, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat_input)

# 모델 컴파일
model = Model([encoder_input, decoder_input], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# STEP 5: 추출적 요약 실험
original_text = data['text'][0]  # 전처리 안 된 원문 사용해야 요약 잘 됨
extractive_summary = summarize(original_text, ratio=0.3)
print("📌 원문:", original_text[:300])
print("✂️ 추출 요약:", extractive_summary if extractive_summary.strip() else "⚠️ 요약 불가")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


📌 원문: Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program and upGrad's 360-degree career support helped him transition to a Data Scientist at Tech Mahindra with 9
✂️ 추출 요약: ⚠️ 요약 불가


In [ ]:
from summa.summarizer import summarize

text = """
South Korea has reported fewer than 200 new COVID-19 cases for the third consecutive day.
Health authorities said that strict social distancing measures are helping to slow the spread.
Vaccination efforts continue, with priority groups receiving their second dose.
Officials urge people to continue wearing masks and avoiding large gatherings.
"""

summary = summarize(text, ratio=0.4)
print("✂️ 요약 결과:\n", summary if summary.strip() else "⚠️ 요약된 내용이 없습니다.")

✂️ 요약 결과:
 Vaccination efforts continue, with priority groups receiving their second dose.
